# ERA5 and SNOTEL
### Download ERA5 and multiple SNOTEL data sites

This is a mix of command line run in the notebook and Python code

In [ ]:
! pip install era5cli && pip install metloom && pip install pyarrow

Add your ID_number and KEY below following the [era5cli documentation](https://era5cli.readthedocs.io/en/stable/getting_started/)

In [ ]:
era5cli config --uid ID_NUMBER --key "KEY"

The cell below has the variables, years, splits out months, for the bounding box around Grand Mesa CO

In [ ]:
! era5cli hourly \
    --land \
    --variables 2m_temperature 2m_dewpoint_temperature u_component_of_wind v_component_of_wind specific_snow_water_content\
    --startyear 2020 \
    --endyear 2022 \
    --splitmonths True \
    --area 39.21 -108.36 38.80 -107.62

### SNOTEL Data
With the ERA5 data downloaded now download the SNOTEL data via Python

In [ ]:
from metloom.pointdata import SnotelPointData
import pandas as pd
from datetime import datetime

GRAND_MESA_SITES = {"Mesa Lakes": "622:CO:SNTL", "Park Reservoir": "682:CO:SNTL"}

ALLOWED_VARIABLES = [
    SnotelPointData.ALLOWED_VARIABLES.SWE,
    SnotelPointData.ALLOWED_VARIABLES.TEMPAVG,
    SnotelPointData.ALLOWED_VARIABLES.SNOWDEPTH,
    SnotelPointData.ALLOWED_VARIABLES.PRECIPITATION,
]


def get_multiple_sites(
    start_date: datetime, end_date: datetime, variables: list, site_dict: dict
) -> pd.DataFrame:
    """
    Get's multiple SNOTEL sites and combines them into one unsorted pandas dataframe
    """
    df = pd.DataFrame()
    for site_name in site_dict.keys():
        single_point = SnotelPointData(station_id=site_dict[site_name], name=site_name)
        print(f"{single_point} download is complete")
        single_data = single_point.get_daily_data(
            start_date=start_date,
            end_date=end_date,
            variables=variables,
        )
        df = pd.concat([df, single_data])
    return df

An example to download the Grand Mesa SNOTEL data for two days in October 2020

In [ ]:
get_multiple_sites(
    datetime(*(2020, 10, 1)),
    datetime(*(2020, 10, 2)),
    ALLOWED_VARIABLES,
    GRAND_MESA_SITES,
)

In [ ]:
gm_df = get_multiple_sites(
    datetime(*(2020, 10, 1)),
    datetime(*(2021, 10, 1)),
    ALLOWED_VARIABLES,
    GRAND_MESA_SITES,
)

In [ ]:
gm_df.to_csv("grand_mesa_snotel.csv")